In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LinearSegmentedColormap
import sys

from scipy.interpolate import lagrange
from scipy.interpolate import interp1d
from scipy.interpolate import spline
from scipy.interpolate import BSpline
from scipy import special
argv = sys.argv
argc = len(argv)
IUNIT = 0 + 1j
Ev_const = 27.21138505

In [3]:
def main():

    w_fname = "../wang.txt"

    fname = 'Ne.00.00.00.05.surff'


    with open(fname, "r") as fp:
        line = fp.readline()
        if len(list(line.strip()))  != 0:
            if list(line.strip())[0] == "#":
                num_krad = int(fp.readline().strip("\n"))
                num_kang = int(fp.readline().strip("\n"))
                skipnum = 3
            else:
                skipnum = 2
                num_krad = int(line.strip("\n"))
                num_kang = int(fp.readline().strip("\n"))

    data = pd.read_csv(fname, comment="#", delim_whitespace=True, header=None, skiprows=skipnum)
    krad = data.iloc[:,0].values
    kang = data.iloc[:,1].values
    cval = data.iloc[:,2:].values
    #print(cval.shape)
    #print(len(data.iloc[0,:]) - 2)

    krad = krad.reshape(num_krad, num_kang)
    kang = kang.reshape(num_krad, num_kang)


    norb = int(len(cval[0,:]) / 2)
    print('norb is', norb)
    xang = np.cos(kang[0, :])

    lmax = 3
    mnum = 3

    mmax = 1
    val = np.zeros((norb, lmax+1, num_krad), dtype=np.complex)

    wdata = pd.read_csv(w_fname, comment="#", delim_whitespace=True, header=None, skiprows=0)
    wang = wdata.iloc[:, 2].values
    xang = wdata.iloc[:, 0].values

    print("# number of orbital : ", norb)
    m_list = [0, 0, -1, 0, 1]
    for iorb in range(norb):
        realp = cval[:,iorb*2].reshape(num_krad, num_kang)
        imagp = cval[:,iorb*2+1].reshape(num_krad, num_kang)

        for irad in range(num_krad):
                for l in range(lmax+1):
                    m = m_list[iorb]
                    if np.abs(m) > l:
                        continue
                    assoc_leg = special.lpmv(np.abs(m), l, xang)
                    """
                    #       ==============================
                    #       this part is missed in my code (td1c, surff_init)
                    """
                    fac = np.sqrt( (2.0*l+1.0) / 2.0 * np.math.factorial(l -  np.abs(m)) / np.math.factorial(l +  np.abs(m)))
                    sph_val = fac * assoc_leg
                    val[iorb, l, irad] = np.sum(wang * sph_val * (realp[irad,:] + IUNIT * imagp[irad,:]))
    m_list2 = [-1, 0, 1]
    val_msum = np.zeros((mnum, lmax+1, num_krad), dtype=np.complex)
    amp = np.zeros((mnum, lmax+1, num_krad))
    phase = np.zeros((mnum, lmax+1, num_krad))
    for im, m in enumerate(m_list2):
        for iorb in range(norb):
            for l in range(lmax+1):
                if m == m_list[iorb]:
#                     print("# im, m, iorb : ",im,", ",  m, ", ", iorb)
                    val_msum[im, l, :] += val[iorb, l, :]

    
    
    amp = np.abs(val_msum)
    phase = np.arctan2(np.imag(val_msum), np.real(val_msum))
    
#   m = -1 is im = 0, m = 0 is im = 1,  m = 1 is im = 2
                
#     max_amp = []    
#     max_phase = []
#     col = [] 
#     for l in range(lmax+1): 
#         for im in range(mnum):
            
#             base = im
#             v2 = val_msum / val_msum[base, 2, :]
#             ph = np.arctan2(np.imag(v2), np.real(v2))

#             df = pd.DataFrame(index = [], columns = [])
#             col_name = "l: "+str(l) + ", " + "m: "+ str(m_list2[im])
#             col_amp = "Amp" + col_name
#             col_phase = "Ph_dif" + col_name
#             df[col_amp]=amp[im, l, 60:80]
#             df[col_phase]=ph[im, l, 60:80]
#             max_amp.append(df[col_amp].max())   
#             max_phase.append(df[col_phase][df[col_amp].idxmax()])
#             col.append(col_name)
#             df.index = 0.5*krad[60:80, 0]**2 * Ev_const
#             df.to_csv('amp_phase.csv', mode='w')

#             df1 = pd.DataFrame({'C_val': max_amp,'Ph_dif': max_phase})
#             df1.index = col
# #     df1.drop(['l: 0, m: -1', 'l: 0, m: 1'])
#     df1.drop(df1.index[[0,2]], inplace=True)
#     df1.to_csv('amp_phase_as_l.csv', mode='w')      
#     print(df1)


    Amp_sheet = np.zeros((mnum, lmax+1))
    Phase_sheet = np.zeros((mnum, lmax+1))

    for l in range(lmax+1): 
        for im in range(mnum):
            if(l == 0 and (im == 0 or im == 2 )):
                continue
           
            base = im
            v2 = val_msum / val_msum[base, 2, :]
            ph = np.arctan2(np.imag(v2), np.real(v2))
            
            col_amp = amp[im, l, 60:80]
            max_amp_pos = np.argmax(col_amp)
            col_phase = ph[im, l, 60:80]
            
            Amp_sheet[im, l] = np.amax(col_amp)
            Phase_sheet[im, l] = col_phase[max_amp_pos]
            
    col_name = ['l = 0', 'l = 1', 'l = 2', 'l = 3']
    df = pd.DataFrame(Amp_sheet, columns = col_name)
    df1 = pd.DataFrame(Phase_sheet, columns = col_name)
    df.index = df1.index =['m = -1', 'm = 0', 'm = 1']

    df['l = 0'][0] = 'NaN'
    df['l = 0'][2] = 'NaN'
    df1['l = 0'][0] = 'NaN'
    df1['l = 0'][2] = 'NaN'

    print(df)
    print(df1)
    
    df.to_csv('Amplitude_sheet.csv', mode = 'w')
    df1.to_csv('Phase_difference_sheet.csv', mode = 'w')



#             col.append(col_name)
#             df.index = 0.5*krad[60:80, 0]**2 * Ev_const
#             df.to_csv('amp_phase.csv', mode='w')

    print(0.5*krad[69, 0]**2 * Ev_const)






if __name__ == "__main__":
    main()



norb is 5
# number of orbital :  5
           l = 0     l = 1     l = 2     l = 3
m = -1       NaN  0.030177  0.025369  0.039723
m = 0   0.019175  0.037664  0.029017  0.048385
m = 1        NaN  0.030177  0.025369  0.039723
           l = 0     l = 1  l = 2     l = 3
m = -1       NaN -2.329838    0.0  1.161638
m = 0   2.338519 -2.153382    0.0  1.162043
m = 1        NaN -2.329838    0.0  1.161638
7.000207426687168
